# Load data

In [3]:
import pandas as pd

data = pd.read_csv("data.csv")
data

,Unnamed: 0,age,gender,country,english,attention_check,d0_condition,d1_condition,d2_condition,d0_q1,...,d2_q5,d2_q6,d2_q7,d2_q8,d2_q9,d2_q10,d2_q11,d2_q12,d2_q13,d2_q14
0,0,25,male,IT,med,"[0, 1, 5]",model1_int,model2_int,model2_ext,1,...,5,2,1,2,3,4,2,1,3,0
1,1,24,female,GR,med,"[0, 1, 5]",model0_ext,model0_ext,model0_int,0,...,3,2,3,3,4,2,6,6,6,0
2,2,31,female,IT,med,"[0, 1, 5]",model0_int,model2_ext,model1_int,2,...,1,4,5,4,3,4,6,5,5,0
3,3,18,female,IT,med,"[0, 1, 5]",model2_int,model1_ext,model2_int,4,...,4,1,2,1,1,1,5,5,4,0
4,4,30,male,FI,med,"[0, 1, 5]",model1_ext,model1_int,model1_ext,0,...,0,1,4,1,3,4,5,5,5,0
5,5,23,female,ZA,high,"[0, 1, 5]",model2_ext,model0_int,model0_ext,0,...,0,6,6,6,6,6,6,6,6,0
6,6,63,female,GR,med,"[0, 5]",model1_int,model2_int,model2_ext,0,...,3,4,3,3,2,5,5,5,3,0
7,7,26,male,GR,med,"[0, 1, 5]",model0_ext,model0_ext,model0_int,3,...,4,3,3,2,2,3,5,4,4,0
8,8,33,female,PL,med,"[0, 1]",model0_int,model2_ext,model1_int,1,...,1,4,4,2,3,3,5,4,5,0
9,9,36,female,ZA,high,[5],model2_int,model1_ext,model2_int,4,...,3,4,2,2,2,2,6,0,4,5


## Remove following attention checks

In [4]:
data = data.drop('Unnamed: 0', axis=1)
data = data[data['d0_q14'] == 0]
data = data[data['d1_q14'] == 0]
data = data[data['d2_q14'] == 0]
data['attention_check'][0]
to_drop = []
list(map(int, data['attention_check'][0][1:-1].split(',')))
for index, row in data.iterrows():
    a_c = list(map(int, row['attention_check'][1:-1].split(',')))
    if 0 in a_c and 1 in a_c:
        continue
    to_drop.append(index)
print(f'Dropped {len(to_drop)} rows')
data = data.drop(to_drop, axis=0)

Dropped 5 rows


In [5]:
from collections import defaultdict
d_0 = defaultdict(lambda: [])
d_1 = defaultdict(lambda: [])
d_2 = defaultdict(lambda: [])
questions = [f'q{i+1}' for i in range(14)]
for _, row in data.iterrows():
    d_0[row['d0_condition']].append([row[f'd0_{q}'] for q in questions])
    d_1[row['d1_condition']].append([row[f'd1_{q}'] for q in questions])
    d_2[row['d2_condition']].append([row[f'd2_{q}'] for q in questions])
print(list(len(value) for value in d_0.values()))
print(list(len(value) for value in d_1.values()))
print(list(len(value) for value in d_2.values()))

[3, 5, 3, 4, 4, 4]
[3, 5, 3, 4, 4, 4]
[3, 5, 3, 4, 4, 4]


In [6]:
import numpy as np

ext = {}
intro = {}
fluency = {}

for i, d in enumerate([d_0, d_1, d_2]):
    for key, value in d.items():
        intro[f'd{i}_{key}'] = np.array(value)[:, :5].mean()
        ext[f'd{i}_{key}'] = np.array(value)[:, 5:10].mean()
        fluency[f'd{i}_{key}'] = np.array(value)[:, 10:13].mean()
print(sorted(ext.items()))
print(sorted(intro.items()))
print(sorted(fluency.items()))

[('d0_model0_ext', 4.72), ('d0_model0_int', 4.333333333333333), ('d0_model1_ext', 5.35), ('d0_model1_int', 4.666666666666667), ('d0_model2_ext', 4.5), ('d0_model2_int', 3.45), ('d1_model0_ext', 5.32), ('d1_model0_int', 3.5), ('d1_model1_ext', 5.2), ('d1_model1_int', 4.6), ('d1_model2_ext', 4.666666666666667), ('d1_model2_int', 4.4), ('d2_model0_ext', 4.7), ('d2_model0_int', 3.04), ('d2_model1_ext', 3.4), ('d2_model1_int', 3.8666666666666667), ('d2_model2_ext', 2.3333333333333335), ('d2_model2_int', 2.95)]
[('d0_model0_ext', 1.76), ('d0_model0_int', 2.7333333333333334), ('d0_model1_ext', 1.35), ('d0_model1_int', 1.6666666666666667), ('d0_model2_ext', 1.3), ('d0_model2_int', 2.35), ('d1_model0_ext', 1.16), ('d1_model0_int', 1.9), ('d1_model1_ext', 0.85), ('d1_model1_int', 1.55), ('d1_model2_ext', 1.8), ('d1_model2_int', 1.7333333333333334), ('d2_model0_ext', 1.15), ('d2_model0_int', 3.16), ('d2_model1_ext', 2.65), ('d2_model1_int', 2.0), ('d2_model2_ext', 3.6666666666666665), ('d2_model2

# Get performance across-dialogues

In [12]:
performance = defaultdict(lambda: [])
for key, value in ext.items():
    performance[key.split("_")[-2]].append(value if 'ext' in key else 6-value)
    if 'ext' in key:
        performance["_".join(key.split("_")[-2:])].append(value)
for key, value in intro.items():
    performance[key.split("_")[-2]].append(value if 'int' in key else 6-value)
    if 'int' in key:
        performance["_".join(key.split("_")[-2:])].append(value)
for key, value in fluency.items():
    performance[f'{key.split("_")[-2]}_fluency'].append(value)
print(sorted(list(map(lambda x: (x[0], np.array(x[1]).mean()), performance.items()))))

[('model0', 3.6325), ('model0_ext', 4.913333333333333), ('model0_fluency', 4.624074074074074), ('model0_int', 2.5977777777777775), ('model1', 3.098611111111111), ('model1_ext', 4.65), ('model1_fluency', 4.430555555555556), ('model1_int', 1.738888888888889), ('model2', 3.0430555555555556), ('model2_ext', 3.833333333333334), ('model2_fluency', 2.902777777777778), ('model2_int', 2.1944444444444446)]


# Performance dialogue 0